# Automating Crypto Website using API

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
#Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {'start':'1', 'limit':'15', 'convert':'USD'}
headers = {'Accepts': 'application/json', 'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509'}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [ ]:
type(data)

In [ ]:
import pandas as pd
#This allows you to see all the columns, not just like 15
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [ ]:
df_ap = pd.json_normalize(data['data'])
df_ap['timestamp'] = pd.to_datetime('now')
df_ap

**Automating**

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
def api_runner():
    global df_ap # declaring 'df_ap' as global for not to end up in error
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {'start':'1', 'limit':'15', 'convert':'USD'}
    headers = {'Accepts': 'application/json', 'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',}

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        
        
    df_ap2 = pd.json_normalize(data['data'])
    df_ap2['timestamp'] = pd.to_datetime('now')
    #df_ap = df_ap.append(df_ap2)
    df_ap = pd.concat([df_ap, df_ap2])

In [ ]:
import os
from time import time 
from time import sleep
for i in range(333): # it runs 333 times a day
    api_runner()
    print('API Runner Completed')
    sleep(60) # sleep for 1 minute 
exit()    

In [ ]:
df_ap

**Saving to Excel**

In [ ]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd 
def api_runner():
    global df_ap # declaring 'df_ap' as global for not to end up in error
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest' 
    #Original Sandbox Environment: 'https://sandbox-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {'start':'1', 'limit':'15', 'convert':'USD'}
    headers = {'Accepts': 'application/json', 'X-CMC_PRO_API_KEY': '0ad53085-1cb2-4eb8-ad9e-3ffbd7e56509',}

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        #print(data)
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)
        
    # In order to prevent redundancy in the output, need to keep it as the original df_ap     
    df_ap3 = pd.json_normalize(data['data'])
    df_ap3['timestamp'] = pd.to_datetime('now')
    df_ap
    
    if not os.path.isfile(r'C:\Users\hp\OneDrive\Documents\Python Tutorials\Csv\API.csv'):
        df_ap.to_csv(r'C:\Users\hp\OneDrive\Documents\Python Tutorials\Csv\API.csv', header = 'column_names')
    else:
        df_ap.to_csv(r'C:\Users\hp\OneDrive\Documents\Python Tutorials\Csv\API.csv', mode = 'a', header = False) # 'a' means append and we don't want header to repeat after the limit, So 'False'

In [ ]:
import os
from time import time 
from time import sleep
for i in range(333): # it runs 333 times a day
    api_runner()
    print('API Runner Completed')
    sleep(60) # sleep for 1 minute 
exit()

In [ ]:
# Reading the csv file to python
df_ap4 = pd.read_csv(r'C:\Users\hp\OneDrive\Documents\Python Tutorials\Csv\API.csv')
df_ap4

In [ ]:
df_ap # printing right here

In [ ]:
pd.set_option('display.float_format', lambda x: '%5f' % x)
df_ap

In [ ]:
df_ap5 = df_ap.groupby("name", sort = False)[df_ap.columns[20:26]].mean()
df_ap5
#df_ap5 = df_ap.groupby('name', sort = False)[['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d']].mean()

In [ ]:
df_ap7 = df_ap5.stack() 
df_ap7

In [ ]:
type(df_ap7)

In [ ]:
df_ap8 = df_ap7.to_frame(name = 'values')
df_ap8

**Creating the index**

In [ ]:
# Creating the index 
df_ap8.count()

In [ ]:
#index = pd.Index(range(90))
#df_ap9 = df_ap8.set_index(index)
#df_ap9

In [ ]:
index = pd.Index(range(90))
df_ap9 = df_ap8.reset_index()
df_ap9

In [ ]:
df_ap10 = df_ap9.rename(columns = {'name': 'Crytocurrency', 'level_1': 'USD_percent_change'})
df_ap10

**Renaming**

In [ ]:
df_ap10['USD_percent_change'] = df_ap10['USD_percent_change'].replace(['quote.USD.percent_change_1h', 'quote.USD.percent_change_24h', 'quote.USD.percent_change_7d', 'quote.USD.percent_change_30d', 'quote.USD.percent_change_60d', 'quote.USD.percent_change_90d'], ['1h', '24h', '7d', '30d', '60d', '90d'])
df_ap10

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.catplot(x ='USD_percent_change', y = 'values', hue = 'Crytocurrency', data = df_ap10, kind = 'point')

In [ ]:
df_ap11 = df_ap[['name', 'quote.USD.price', 'timestamp']]
df_ap11

In [ ]:
pip install --upgrade seaborn pandas

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
#with pd.option_context('mode.use_inf_as_null', True):
sns.lineplot(x = 'timestamp', y = 'quote.USD.price', data = df_ap11)